# Modeling of Tranquillitatis Pit and Marius Hills Lava Tube

# Converting zero-thickness .obj mesh into a 3D volume and importing it into COMSOL
### 1) Autodesk Fusion
>#### a) Open the .obj file in Autodesk Fusion 360 by clicking on the "Insert" drop down menu in the tool bar and selecting "Insert Mesh". Orient Mesh and select units as needed, in this case our model will lie on the X-Z Plane and be in meters.
#### b) If the provided mesh is too large and has a lot of un-needed space right click on the mesh and select edit, click on the "Modify" drop down menu and select delete faces. Use the Brush to select all of the faces you want to delete and press "OK". Leave some wiggle room on the edge, this is just to save your computer for exploding if the mesh is very large.
#### c) OPTIONAL: If you later find out that the mesh has build errors or there are high mesh density mesh areas that you want to get rid of, it may be useful to use the "Remesh" feature under the Modify drop down menu and let the program remesh it. I personally keept the density at 1 and moved the shape preservation to 0.9.
#### d) Select "Finish Mesh" on the toolbar and save the mesh as a new .obj by right clicking on the mesh and selecting "Save as OBJ".

### 2) Autodesk Inventor
## IMPORTANT : By this point make sure you have downloaded and enabled the "Mesh Enabler" addon for Inventor which can be found by googling "Autodesk Inventor Mesh Enabler" and selecting the first result (URL should be apps.Autodesk.etc.)
>#### e) Open the mesh you want to work with in Autodesk inventor just by opening the .obj file.
#### f) Right click on the mesh name in the design tree (left side and will display the .obj file name) and select "Convert to Base Feature", selecting the cube (solid model) as the output type.
#### g) Make reference planes from the base XY and ZY planes that nearly to the edge of the mesh. This can be done by selecting "Plane" under the feature tab and selecting "Offset from Plane". You can select the base XY and ZY planes by opening the "Origin" folder in the design Tree. Make sure that the area enclosed by the 4 new planes (one on each side) doesnt have any gaps from the jaggedy mesh edge. Not doing this will stop the mesh from becoming a solid later.
#### h) On each of the new planes start a 2D sketch. First you want to "Project Cut Edges" to make a line of where the mesh intersects the plane. After this you will want to put in two lines going down, starting at either edge of the previously made line. Make sure these two lines go down further than the lowest point of the mesh and end at the same height. Connect the bottom of those two lines with another line, click "Patch" on the 3D Model tab in the "Surface" section and select the inside of the shape you made. Its okay if these new surfaces overhang eachother, we can trim them later.
#### i) At this point you should have surfaces on all four sides of the mesh, now we need one on the bottom to close off the entire surface. the can be done by making another reference plane from the base XZ plane that goes down to where the side surfaces end. Once you do this you can make a 2D sketch and use the "Convert Entities" feature, selecting the bottom line of all of the side surfaces, making a box. You can now use the "Patch" feature on the box.
#### j) This may take some tries, but after some trial and error you will be able to do this to all of the overhanging surfaces. Using the "Trim" tool in the 3D Model tab under surfaces you can trim the overhanging surfaces. This is done by selecting one surface as the trimming tool, and another surface that is to be trimmed. Do this for every surface until it is a box with no overhanging parts that arent attached to anything.
#### k) If everything has been done correctly you should have your mesh and a box of surfaces enclosing the underside of it with no overhanging edges. Now you can use the "Stitch" tool in the 3D model tab under surfaces and select the mesh and every surface you made. Pressing OK should make everything into a single surface, and since it is a fully enclosed surface it should generate a 3D volume from the enclosed area.
#### l) Save the new volume as a .ipt, this file type can be imported into comsol with higher accuracy than a .stl or a 3D .obj.

### 3) COMSOL
>#### m) Once COMSOL has been opened you can press the "Import" button in the Geometry section and select the .ipt, Inventor or Autodesk Livelink *may* need to be installed for it to recognize that file type. Import the file in the correct units, the orientation can be changed to fit the needs of the model.

# Modeling solar motion for any given location on the Moon.

## This section is a desription of how to use JPL Horizon's Ephemeris to get the sun's motion in the Lunar sky for any location with only one set of data (Sun location from Lunar South Pole), but this should work for any two objects and any location on the "Observer" Body. The reason we are doing it this way is to make a general model where you input the Mesh's Lunar Latitude and Longitude and that is essentially it, instead of having to import a fresh new Ephemeris data set for every unique location.

## 1) Importing Ephemeris Data for the Lunar South Pole
>### a) https://ssd.jpl.nasa.gov/horizons.cgi will allow you to essentially any data you would like relating two solar system bodies, in this case we will want to know the Sun's location as viewed from (0E, -90N) on the Moon.
### b) The Selections we will be using are as follows,
>>####  Ephemeris Type : Observer Table
>>####  Target Body: Sol
>>####  Observer Location: 
>>>#### Longitude: 0, E 
>>>#### Latitude: 90, S
>>>#### Altitude: 0
>>>#### Body ID: 301

>>####  Time Span: 
>>>#### Start Time: 2018-03-01
>>>#### End Time: 2019-03-01
>>>#### Step Size: 15, Minutes

>>####  Table Settings:
>>>#### Quantities: 4, 20
>>>#### Angle Format: decimal degrees
>>>#### Output Units: AU & AU/d
>>>#### Extra Precision: Yes
>>>#### CSV Format: Yes

>### c) This will output a csv table that has 15 minute interval data of the Sun's location as seen from the Lunar South Pole, this is essentially the sub-solar latitude and longitude. Using this data will allow us to calculate the Sun's Altitude and Azimuth for any location on the Moon as a function of time.
>### d) We can save this output as a .txt and import it into Excel as a CSV, from there we can convert the Sun's Altitude and Azimuth into radians, COMSOL uses radians for trigonometric functions, and import them as well as the distance in AU into COMSOL as interpolations. This will create graphs of each so that any time interval can be modeled accurately, not just 15-minute intervals as generated with the Ephemeris.
>>### Note: The imported time column in excel was changed to seconds instead of date-time so the data would read in easier to COMSOL

>### e) At this point we have Altitude, Azimuth, and Distance in COMSOL that are ready to be used. We now create two parameters, Longitude and Latitude, these will be the location of the mesh in question. Longitude will need to be in radians East (can be input as degrees * 2 * pi / 360 for accuracy) and Latitude will need to be in radians North (positive or negative, also can be input as degrees * 2 * pi / 360 for accuracy)
>### f) Now we will create the functions LocalAltitude and LocalAzimuth to reflect the suns position for any given time at the given Longitude and Latitude. The functions below are what will calculate these values.

\begin{equation}
Local Azimuth(t) = \pi - \frac{sin(Az(t)-Lon(t))}{|sin(Az(t)-Lon(t))|}*\frac{\pi}{2} + tan^{-1}(\sqrt{1-sin(Az(t)-Lon(t))^2cos(Alt(t))^2}*cos(\frac{atan( cos(Az(t)-Lon(t))*cos(Alt(t))*sin(Alt(t)))-Lat(t))}{sin(Az(t)-Lon(t))cos(Alt(t))})
\end{equation}

\begin{equation}
Local Altitude(t) = \frac{\pi}{2} - cos^{-1}(cos(Lat(t))cos(Alt(t))cos(Lon(t)-Az(t))-sin(Lat(t))sin(Alt(t)))
\end{equation}

>### g) While these are useful values to know, it will make it a tad bit annoying to use for calculating the x, y, and z locations of the sun, so we will make another function to get Phi and Theta for a spherical coordinate system, then x, y, and z will be easy to calculate. For this we will also need to make sure the orientation of the mesh is as follows.
>>#### +x : South
>>#### -x : North
>>#### +y : West
>>#### -y : East
>>#### +z : Zenith
>>#### -z : Nadir

\begin{equation}
\theta(t) = 180 + \frac{Local Azimuth(t) - \pi}{|Local Azimuth(t) - \pi|}*180 -Local Azimuth(t) + \pi
\end{equation}

\begin{equation}
\phi(t) = Local Altitude(t) + \frac{\pi}{2}
\end{equation}

>### h) Having these values will allow us to calculate the x, y, and z locations of the sun, these will be input into the Sun's locations when using the infinite distance option, I will also Include the Solar flux function here.

\begin{equation}
x(t) = cos(\theta(t))sin(\phi(t))
\end{equation}

\begin{equation}
y(t) = -sin(\theta(t))sin(\phi(t))
\end{equation}

\begin{equation}
z(t) = cos(\phi(t))
\end{equation}

\begin{equation}
Flux(t) = \frac{1367.7}{Dist(t)^2}
\end{equation}